# Module 7: Interactive Data Console

## Learning Objectives
- Build a Rails console-like interface
- Create interactive data exploration tools
- Implement REPL for data manipulation
- Practice real-time data analysis

## Topics Covered
- Interactive Python shells and REPLs
- Data exploration interfaces
- Real-time data analysis
- Custom command-line tools
- Database query interfaces

## Time Estimate: 2-3 weeks
## Success Criteria: Build a fully functional data exploration console

---

## Section 1: Interactive Python Shells

Creating an interactive console similar to Rails console for data exploration and manipulation.


In [ ]:
# Interactive Data Console Implementation

import sys
import os
from typing import Any, Dict, List, Optional
from datetime import datetime
import json
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import cmd
import readline

class DataConsole(cmd.Cmd):
    """Interactive data console similar to Rails console"""
    
    intro = """
    🐍 Interactive Data Console
    =========================
    
    Welcome to the Data Console! Type 'help' for available commands.
    
    Available commands:
    - help: Show this help message
    - exit/quit: Exit the console
    - db: Database operations
    - data: Data manipulation
    - query: Execute SQL queries
    - load: Load data from files
    - save: Save data to files
    - stats: Show data statistics
    """
    
    prompt = "data_console> "
    
    def __init__(self):
        super().__init__()
        self.db_engine = None
        self.session = None
        self.data_cache = {}
        self.setup_database()
    
    def setup_database(self):
        """Setup database connection"""
        try:
            # Use SQLite for demo
            db_path = "data/console_demo.db"
            os.makedirs("data", exist_ok=True)
            
            self.db_engine = create_engine(f"sqlite:///{db_path}")
            SessionLocal = sessionmaker(bind=self.db_engine)
            self.session = SessionLocal()
            
            # Create sample tables
            self.create_sample_tables()
            print("✅ Database connection established")
            
        except Exception as e:
            print(f"❌ Database setup failed: {e}")
    
    def create_sample_tables(self):
        """Create sample tables for demonstration"""
        create_tables_sql = """
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            age INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            category TEXT,
            stock INTEGER DEFAULT 0
        );
        
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            total REAL,
            order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users(id),
            FOREIGN KEY (product_id) REFERENCES products(id)
        );
        """
        
        with self.db_engine.connect() as conn:
            conn.execute(text(create_tables_sql))
            conn.commit()
        
        # Insert sample data
        self.insert_sample_data()
    
    def insert_sample_data(self):
        """Insert sample data"""
        sample_data = """
        INSERT OR IGNORE INTO users (name, email, age) VALUES
        ('Alice Johnson', 'alice@example.com', 28),
        ('Bob Smith', 'bob@example.com', 35),
        ('Charlie Brown', 'charlie@example.com', 42);
        
        INSERT OR IGNORE INTO products (name, price, category, stock) VALUES
        ('Laptop', 999.99, 'Electronics', 10),
        ('Mouse', 29.99, 'Electronics', 50),
        ('Keyboard', 79.99, 'Electronics', 25),
        ('Book', 19.99, 'Education', 100);
        
        INSERT OR IGNORE INTO orders (user_id, product_id, quantity, total) VALUES
        (1, 1, 1, 999.99),
        (1, 2, 2, 59.98),
        (2, 3, 1, 79.99),
        (3, 4, 3, 59.97);
        """
        
        with self.db_engine.connect() as conn:
            conn.execute(text(sample_data))
            conn.commit()
    
    def do_db(self, args):
        """Database operations"""
        if not args:
            print("Available db commands:")
            print("  db tables - List all tables")
            print("  db schema <table> - Show table schema")
            print("  db count <table> - Count records in table")
            return
        
        parts = args.split()
        command = parts[0]
        
        if command == "tables":
            self.list_tables()
        elif command == "schema" and len(parts) > 1:
            self.show_schema(parts[1])
        elif command == "count" and len(parts) > 1:
            self.count_records(parts[1])
        else:
            print("Unknown db command. Use 'db' for help.")
    
    def list_tables(self):
        """List all database tables"""
        try:
            with self.db_engine.connect() as conn:
                result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table'"))
                tables = [row[0] for row in result]
                print("📊 Available tables:")
                for table in tables:
                    print(f"  - {table}")
        except Exception as e:
            print(f"❌ Error listing tables: {e}")
    
    def show_schema(self, table_name):
        """Show table schema"""
        try:
            with self.db_engine.connect() as conn:
                result = conn.execute(text(f"PRAGMA table_info({table_name})"))
                columns = result.fetchall()
                print(f"📋 Schema for table '{table_name}':")
                for col in columns:
                    print(f"  - {col[1]} ({col[2]}) {'NOT NULL' if col[3] else 'NULL'}")
        except Exception as e:
            print(f"❌ Error showing schema: {e}")
    
    def count_records(self, table_name):
        """Count records in table"""
        try:
            with self.db_engine.connect() as conn:
                result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
                count = result.fetchone()[0]
                print(f"📊 Table '{table_name}' has {count} records")
        except Exception as e:
            print(f"❌ Error counting records: {e}")
    
    def do_query(self, args):
        """Execute SQL query"""
        if not args:
            print("Usage: query <SQL statement>")
            return
        
        try:
            with self.db_engine.connect() as conn:
                result = conn.execute(text(args))
                
                # Check if it's a SELECT query
                if args.strip().upper().startswith('SELECT'):
                    rows = result.fetchall()
                    if rows:
                        # Convert to DataFrame for better display
                        df = pd.DataFrame(rows, columns=result.keys())
                        print(df.to_string(index=False))
                    else:
                        print("No results found")
                else:
                    conn.commit()
                    print("✅ Query executed successfully")
                    
        except Exception as e:
            print(f"❌ Query error: {e}")
    
    def do_data(self, args):
        """Data manipulation commands"""
        if not args:
            print("Available data commands:")
            print("  data load <name> <query> - Load data from query into cache")
            print("  data show <name> - Show cached data")
            print("  data stats <name> - Show data statistics")
            return
        
        parts = args.split(None, 1)
        command = parts[0]
        
        if command == "load" and len(parts) > 2:
            self.load_data(parts[1], parts[2])
        elif command == "show" and len(parts) > 1:
            self.show_data(parts[1])
        elif command == "stats" and len(parts) > 1:
            self.show_stats(parts[1])
        else:
            print("Unknown data command. Use 'data' for help.")
    
    def load_data(self, name, query):
        """Load data from query into cache"""
        try:
            with self.db_engine.connect() as conn:
                result = conn.execute(text(query))
                df = pd.DataFrame(result.fetchall(), columns=result.keys())
                self.data_cache[name] = df
                print(f"✅ Loaded {len(df)} records into '{name}'")
        except Exception as e:
            print(f"❌ Error loading data: {e}")
    
    def show_data(self, name):
        """Show cached data"""
        if name not in self.data_cache:
            print(f"❌ Data '{name}' not found in cache")
            return
        
        df = self.data_cache[name]
        print(f"📊 Data: {name} ({len(df)} records)")
        print(df.head(10).to_string(index=False))
        if len(df) > 10:
            print(f"... and {len(df) - 10} more records")
    
    def show_stats(self, name):
        """Show data statistics"""
        if name not in self.data_cache:
            print(f"❌ Data '{name}' not found in cache")
            return
        
        df = self.data_cache[name]
        print(f"📈 Statistics for '{name}':")
        print(f"  Records: {len(df)}")
        print(f"  Columns: {len(df.columns)}")
        print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
        
        # Show numeric column statistics
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            print("\n📊 Numeric columns summary:")
            print(df[numeric_cols].describe().to_string())
    
    def do_exit(self, args):
        """Exit the console"""
        print("👋 Goodbye!")
        return True
    
    def do_quit(self, args):
        """Exit the console"""
        return self.do_exit(args)
    
    def default(self, line):
        """Handle unknown commands"""
        print(f"Unknown command: {line}")
        print("Type 'help' for available commands")

# Create and run the console
def run_console():
    """Run the interactive data console"""
    console = DataConsole()
    console.cmdloop()

print("✅ Interactive Data Console created!")
print("Run 'run_console()' to start the interactive console")
